In [1]:
import scanpy as sc
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# Data Prepocessing, Normalize, Dimension Reduction

In [ ]:
adata = sc.read_mtx("SRR13482552/matrix.mtx")
adata = adata.T
gene = pd.read_table("SRR13482552/features.tsv", header = None)
gene = np.array(gene[1])
barcode = pd.read_table("SRR13482552/barcodes.tsv", header = None)
barcode = np.array(barcode[0])

In [ ]:
adata.obs_names = barcode
adata.var_names = gene
adata

In [ ]:
adata.var_names_make_unique()
adata.obs

# Read h5ad file

In [ ]:
raw = sc.read_h5ad("../../PBMC_Data/merge_done/GSE165080/Raw.h5ad")

In [ ]:
raw.obs

In [ ]:
raw

In [ ]:
raw.var_names_make_unique()
raw.obs_names_make_unique()
sc.pp.filter_cells(raw, min_genes=200)
sc.pp.filter_genes(raw, min_cells=3)

In [ ]:
raw

In [ ]:
raw.var['mt'] = raw.var_names.str.startswith('MT-')
sc.pp.calculate_qc_metrics(raw, qc_vars=['mt'], percent_top=None, log1p=False, inplace=True)

In [ ]:
sc.pl.violin(raw, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(raw, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(raw, x='total_counts', y='n_genes_by_counts')

In [ ]:
raw = raw[raw.obs.n_genes_by_counts < 6000, :]
raw = raw[raw.obs.pct_counts_mt < 25, :]
raw

In [ ]:
sc.pl.violin(raw, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'],
             jitter=0.4, multi_panel=True)

In [ ]:
sc.pp.normalize_total(raw, target_sum=1e4)

In [ ]:
sc.pp.log1p(raw)
raw.raw = raw
raw

In [ ]:
# linear regression out unwanted sources of variation
sc.pp.regress_out(raw, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.scale(raw, max_value=10)
sc.tl.pca(raw, svd_solver='arpack')
sc.pl.pca_variance_ratio(raw, log=True)

In [ ]:
sc.pp.neighbors(raw, n_neighbors=10, n_pcs=40)
sc.tl.leiden(raw)

In [ ]:
# default use leiden透過leiden去計算那個gene expression的部分去挑出最後的權重
#再多維的空間做hireachecal(階層的)挑選，clustering會越來越少，直到hirachical跟cluster一樣才會結束

sc.tl.paga(raw)
sc.pl.paga(raw)

In [ ]:
sc.tl.umap(raw, init_pos='paga')
sc.pl.umap(raw, color = "leiden")

In [ ]:
raw.write_h5ad("Data/GSE165080_healthy.h5ad")

# 讀取已經寫好的檔案

In [2]:
raw = sc.read_h5ad("Data/GSE165080_healthy.h5ad")

In [3]:
raw

AnnData object with n_obs × n_vars = 46160 × 27426
    obs: 'Age', 'Gender', 'BioSample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden'
    var: 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'mean', 'std'
    uns: 'leiden', 'leiden_colors', 'leiden_sizes', 'log1p', 'neighbors', 'paga', 'pca', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

# 統整cells and genes的數量

In [ ]:
cells_and_genes = {}

for i in range(0, Raw.n_obs):
    cells_and_genes[Raw.obs.BioSample[i]] = []
for i in cells_and_genes:
    healthy = Raw[Raw.obs["BioSample"] == i, :]
    cells_and_genes[i] = [f"{healthy.n_obs}*{healthy.n_vars}"]
    print(i)

In [ ]:
cells_and_genes

# 紀錄檔案的性別、年齡、資料集

In [ ]:
adata.obs["Age"] = [44]*adata.n_obs
adata.obs["Gender"] = ["female"]*adata.n_obs
adata.obs["Dataset"] = ["SRR13482552"]*adata.n_obs
adata.obs

In [ ]:
adata.write_h5ad("Healthy.h5ad")

# 以下是GSE149689所需的code

In [ ]:
# barcode
# unfilter data
pre = adata.obs_names

# extract Healthy Donor from barcode
# 這些5 13 14 19是GSE149689中表示healthy donor
# $是結尾
pattern = r"-(5|13|14|19)$"
pre_healthy = pre[pre.str.contains(pattern)]


pre_healthy

In [ ]:
healthy = adata[adata.obs.index.isin(pre_healthy)]
healthy.X

In [ ]:
sc.pp.normalize_total(healthy, target_sum=1e4)

In [ ]:
sc.pp.log1p(healthy)
healthy.raw = healthy
healthy

In [ ]:
# linear regression out unwanted sources of variation
sc.pp.regress_out(healthy, ['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pp.scale(healthy, max_value=10)
sc.tl.pca(healthy, svd_solver='arpack')
sc.pl.pca_variance_ratio(healthy, log=True)

In [ ]:
sc.pp.neighbors(healthy, n_neighbors=10, n_pcs=40)
sc.tl.leiden(healthy)

In [ ]:
# default use leiden透過leiden去計算那個gene expression的部分去挑出最後的權重
#再多維的空間做hireachecal(階層的)挑選，clustering會越來越少，直到hirachical跟cluster一樣才會結束

sc.tl.paga(healthy)
sc.pl.paga(healthy)

In [ ]:
sc.tl.umap(healthy, init_pos='paga')
sc.pl.umap(healthy, color = "leiden")

In [ ]:
healthy.write_h5ad("benson_pbmc_healthy_donor.h5ad")

In [ ]:
sc.pl.umap(healthy, color = "leiden")

In [ ]:
healthy